In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/My\ Drive/stylegan-refactor

/content/gdrive/My Drive/stylegan-refactor


In [4]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-nouc8q6g
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-nouc8q6g
  Stored in directory: /tmp/pip-ephem-wheel-cache-f16rbpjj/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [13]:
%cd /content/gdrive/My\ Drive/stylegan-refactor
!python encoder.py -i ./aligned_realpics/64/chicago2.png -n l2only --img_size 64 --layersF 1 5 --steps 700 --loss 1*L2+0.0075*REG+0.0075*CROSS --lr 0.1 --optimizer ADAM --mask_type SEGMENT

/content/gdrive/My Drive/stylegan-refactor
Using TensorFlow backend.
using -n for output
Working on chicago2.png
Built mask from image
L2: 0.097/0.093 REG: 3.989/3.814 CROSS: 6.652/5.928, TOTAL: 0.177/0.166, BEST_IDX: 685, LR: 0.10000: 100% 700/700 [02:13<00:00,  5.87it/s]


In [0]:
%cd /content/gdrive/My\ Drive/stylegan-refactor
!cat layerloop.sh

In [0]:
%cd /content/gdrive/My\ Drive/stylegan-refactor
!bash layerloop.sh

In [62]:
from pathlib import Path
from keras.preprocessing import image
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import PIL.Image
from bicubic_downsample import Downsampler

def load_image(im, img_size): 
    img = image.load_img(im)
    img = np.expand_dims(img, 0)
    sess = tf.Session()
    D = Downsampler()
    with sess.as_default():
      img = tf.cast(img,dtype='float32')
      img = D.resize_bicubic(img, img_size)
      img = tf.saturate_cast(img,dtype='uint8')
      img = img.eval()
    return img[0]
  
SIZE=64;
  
for im_path in Path().glob("aligned_realpics/*.png"):
  out_file = Path("aligned_realpics")/str(SIZE)/im_path.name
  out_file.parent.mkdir(parents=True,exist_ok=True)
  if(out_file.is_file()):
    print(f"Found {im_path.name}")
  else:
    img = load_image(im_path,SIZE)
    img = PIL.Image.fromarray(img)
    img.save(out_file)
    print(f"Saved {im_path.name}")
  

Using TensorFlow backend.


Found chicago3.png
Found ObamaHR.png
Found chicago1.png
Found chicago2.png
Found chicago6.png
Found chicago4.png
Found chicago5.png
Found keanu2.png
Found keanu.png
Saved tpdne.png


In [1]:
%cd /content/gdrive/My\ Drive/stylegan-refactor
import logging
logging.getLogger('tensorflow').disabled = True

import os
import sys
from pathlib import Path
from datetime import datetime
import glob

file_path = Path().resolve()
print("Current Path: " + str(file_path))

last_edit,last_file = max([(os.stat(filename).st_mtime,filename) for filename in Path().glob('**/*.py')])
last_edit = datetime.fromtimestamp(last_edit)
current_time = datetime.now()
diff_time = current_time-last_edit
print("Last python file (" + str(last_file) + ") was modified " + str(diff_time.total_seconds())+" seconds ago")

import argparse
import pickle
from tqdm import tqdm
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from tensorflow.train import cosine_decay_restarts
from matplotlib import pyplot as plt
import scipy.misc
from SR.generator import Generator
from SR.optimizer import SROptimizer
from tqdm import trange

URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'  # karras2019stylegan-ffhq-1024x1024.pkl

def get_args():
    parser = argparse.ArgumentParser(description='Style-Gan Super Resolution')
    parser.add_argument('-i', '--input_files', help='Input files')
    parser.add_argument('-o', '--output_dir', default="generated_images/", help='Directory for storing output images and latents')
    parser.add_argument('-n', '--output_name', default=None, help='Name to call output file')
    parser.add_argument('--loss_dir', default="Loss/", help='Directory for storing loss logs')
    parser.add_argument('--n_init', default=1, help='Number of initializations for encoder', type=int)
    parser.add_argument('--img_size', default=64, help='Size to rescale to', type=int)
    parser.add_argument('--loss', default='1.0*L2', help='Loss function to use')
    parser.add_argument('-LIN','--layersIN', default=[3,6,9], nargs='+',help='Which VGG-ImageNet layers to use',type=int)
    parser.add_argument('-LF','--layersF', default=[3,6,9], nargs='+',help='Which VGG-Face layers to use',type=int)
    parser.add_argument('--lr', default=1., help='Learning rate', type=float)
    parser.add_argument('--optimizer', default='SGD', help='Which optimizer to use')
    parser.add_argument('--cosine_cycle', default=None, help='Whether to use cosine annealing',type=int)
    parser.add_argument('--steps', default=1500, help='Number of gradient-descent steps', type=int)
    parser.add_argument('--mask_type', default=None, help='Whether to weight the pixels differently with a face mask')

    args,other_args = parser.parse_known_args()
    if(len(other_args)>0): print(f"Did not recognize arguments: {other_args}")
    return args

def plot_loss(losslist,best_losses,loss_dir):
    losses = np.array(losslist)
    axis = np.array(range(len(losses)))
    label = 'Loss'
    fig = plt.figure()
    plt.title(label)
    plt.plot(axis, losses, label="Loss",zorder=1)
    plt.scatter(axis[best_losses], losses[best_losses], s=5, c='r', label="Best",zorder=2)
    plt.legend()
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.savefig(loss_dir/'loss.pdf')
    plt.close(fig)

sys.argv = [sys.argv[0],
              '-i', './aligned_realpics/chicago2.png',
              '-n', "layer?.png",
              '--img_size', '64', 
              '--layersF', "50", 
              '--loss', '1*FACE+0.04*REG+0.02*CROSS', 
              '--lr', '0.1', 
              '--steps', '50',
              '--optimizer', 'ADAM', 
              '--mask_type', 'SEGMENT']
    
args = get_args()

ref_images = Path().glob(args.input_files)
# ref_images = list(ref_images)

output_dir = Path(args.output_dir)
output_dir.mkdir(exist_ok=True)
loss_dir = Path(args.loss_dir)
loss_dir.mkdir(exist_ok=True)

# Load StyleGAN
print("Loading StyleGAN")
tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)

print("Loaded StyleGAN")

print("Creating Generator")
G = Generator(Gs_network, args)
G.reset_latent()
print("Created Generator")
print("Creating Optimizer")
opt = SROptimizer(G, args)
print("Created Optimizer")

for i in range(1,18):
  args.output_name = f"layer{i}.png"
  opt.layersF = [i]

  print("Building Loss")
  opt.build_loss()
  print("Built Loss")

  losses=[]
  best_losses=[]

  for image in ref_images:
      print(f"Working on {image.name}")
      opt.set_reference_image(image)

      image_mask = opt.image_mask.eval()[0]
      image_mask = np.clip(255*image_mask,0,255).astype('uint8')
      image_mask = PIL.Image.fromarray(image_mask)
      image_mask.save(output_dir/'mask.png')

      t = trange(args.steps)

      min_loss = np.inf
      best_img = None
      best_idx = 0
      best_list = [np.inf,np.inf,np.inf]
      cos_iter = 0
      cos_cycle = args.cosine_cycle

      for i in t:
          current_lr = args.lr
          if(cos_cycle is not None):
              current_lr = 1e-5 + 0.5 * (args.lr - 1e-5) * \
               (1. + np.cos(cos_iter * np.pi / cos_cycle))
              cos_iter += 1
              if(cos_iter == cos_cycle):
                  cos_iter = 0
                  cos_cycle = 2*cos_cycle
          loss_list,loss = opt.step(lr=current_lr)
          losses.append(loss)
          curr_str = ' '.join([f"{x[0]}: {x[1]:.3f}/{y:.3f}" for x,y in zip(loss_list,best_list)])
          curr_str += f", TOTAL: {loss:.3f}/{min_loss:.3f}, BEST_IDX: {best_idx}, LR: {current_lr:.5f}"

          if(loss < min_loss):
              min_loss = loss
              best_losses.append(i)
              best_idx = i
              best_list = [x[1] for x in loss_list]
              if(i>=0.1*args.steps):
                  generated_image = G.generate_images()
                  generated_latent = G.latent.eval()
                  best_img = PIL.Image.fromarray(generated_image[0], 'RGB')
                  best_latent=generated_latent

          t.set_description(curr_str)

          if(i%100 == 0):
              if(best_img is not None):
                  best_img.save(output_dir / image.name, 'PNG')
                  np.save(output_dir / image.stem, best_latent)
              plot_loss(losses,best_losses,loss_dir)

      if(args.output_name is None):
          output_name = image.stem
      else:
          output_name = args.output_name

      best_img.save(output_dir / f'{output_name}.png', 'PNG')
      np.save(output_dir / f'{output_name}.npy', best_latent)
      plot_loss(losses,best_losses,loss_dir)

/content/gdrive/My Drive/stylegan-refactor
Current Path: /content/gdrive/My Drive/stylegan-refactor
Last python file (SR/optimizer.py) was modified 1666.358866 seconds ago


Using TensorFlow backend.


Loading StyleGAN
Loaded StyleGAN
Creating Generator
Created Generator
Creating Optimizer
Created Optimizer
Building Loss
 > Building FACE
 > > Using layers [1] for face loss
 > Building REG
 > Building CROSS
Built Loss
Working on chicago2.png
Built mask from image


FACE: 0.542/0.568 REG: 2.598/2.640 CROSS: 10.476/10.761, TOTAL: 0.855/0.888, BEST_IDX: 47, LR: 0.10000: 100%|██████████| 50/50 [00:15<00:00,  5.01it/s]


Building Loss


ValueError: ignored